![Earth](images_notebk/vaca_dream.jpg)

In [1]:
%matplotlib inline

In [2]:
# Dependencies and Setup
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
city_data = pd.read_csv("output_data/cities.csv")
city_data['City']=city_data['City'].str.title()
coordinates=city_data[['Lat', 'Lng']]
weight = city_data['Humidity']
max_intensity = city_data['Humidity'].max()
city_data.head()

,Index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Arlit,18.7369,7.3853,94.03,15,47,5.88,NE,1624484744
1,1,Thompson,55.7435,-97.8558,62.76,59,75,13.80,CA,1624484707
2,2,Saint George,37.1041,-113.5841,94.62,20,1,8.01,US,1624484461
3,3,Cernatesti,45.2667,26.7667,67.80,76,99,5.10,RO,1624484744
4,4,Vaini,-21.2000,-175.2000,68.16,88,75,4.61,TO,1624484745


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
figure_layout = {"width": "625","height": "500px","border": "1px solid black","padding": "1px","margin": "0 auto 0 auto"}
fig = gmaps.figure(layout=figure_layout)

fig.add_layer(gmaps.heatmap_layer(coordinates, weights=weight, max_intensity=max_intensity, point_radius=6, opacity=0.8))
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# A max temperature lower than 80 degrees but higher than 70. 
# I dont mind a few degrees cooler, so going with max temps bwteeen 68 and 80
# Wind speed less than 10 mph.
# Zero cloudiness.

#Store into variable named hotel_df
hotel_df = pd.DataFrame(city_data.loc[((city_data["Max Temp"]>=68) & (city_data["Max Temp"]<=80)) & (city_data["Wind Speed"]<10) & (city_data["Cloudiness"] == 0) ])

# Add a "Hotel Name" column to the DataFrame.
hotel_df['HotelName'] = ""
hotel_df.head()

,Index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,HotelName
21,21,Arraial Do Cabo,-22.9661,-42.0278,69.76,78,0,3.44,BR,1624484750,
54,54,Martil,35.6166,-5.2752,71.67,78,0,6.91,MA,1624484760,
80,80,Gasa,31.5000,34.4667,76.98,74,0,4.23,PS,1624484768,
84,84,Korla,41.7597,86.1469,71.49,21,0,8.55,CN,1624484769,
259,259,Uttarkashi,30.7333,78.4500,68.50,72,0,6.42,IN,1624484891,


In [6]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# use iterrows to iterate through hotel_df dataframe
# Set parameters to search each hotel within 5000 meters.
for index, row in hotel_df.iterrows():
    locations = str(row["Lat"])+","+str(row["Lng"])
    params = {
        "location": locations, 
        "radius": '5000',
        "type": "lodging",
        "key": g_key,
    }
    # Hit the Google Places API for each city's coordinates.
    response = requests.get(base_url, params=params).json()

    #Store the first Hotel result into the DataFrame.
    hotel_df.loc[index, 'HotelName'] = (response['results'][0]['name']).title()

In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt><b>Name</b></dt><dt>{HotelName}</dt>
<dt><b>City</b></dt><dt>{City}</dt>
<dt><b>Country</b></dt><dt>{Country}</dt>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
hotel_df

,Index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,HotelName
21,21,Arraial Do Cabo,-22.9661,-42.0278,69.76,78,0,3.44,BR,1624484750,Pousada Porto Praia
54,54,Martil,35.6166,-5.2752,71.67,78,0,6.91,MA,1624484760,فندق مرتيل
80,80,Gasa,31.5000,34.4667,76.98,74,0,4.23,PS,1624484768,رابطة الفنانين الفلسطينيين
84,84,Korla,41.7597,86.1469,71.49,21,0,8.55,CN,1624484769,Gulisitan Hotel
259,259,Uttarkashi,30.7333,78.4500,68.50,72,0,6.42,IN,1624484891,Anand Ganga
271,271,Langarud,37.1970,50.1535,75.18,52,0,4.03,IR,1624484895,ستاد اسکان فرهنگیان لنگرود
305,305,Ukholovo,53.7929,40.4892,70.25,79,0,3.67,RU,1624484905,Pupchiny
394,394,Komsomolskiy,40.4272,71.7189,70.52,35,0,6.91,UZ,1624484996,Ikathouse
429,429,Gat,31.6100,34.7642,75.88,79,0,3.09,IL,1624485008,Olushome
454,454,Sulina,45.1559,29.6536,69.37,89,0,6.85,RO,1624485015,Hotel Delta Est


In [8]:
# Plot markers on top of the heatmap.
markers = gmaps.marker_layer(locations, info_box_content=hotel_info,  hover_text='Click for Hotel')

# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…